### Question 1

Suppose you are a portfolio manager and you are going to use derivatives to construct certain portfolio. The current stock price for AAPL is $\$171.01 per share, at 10 am, March 8, 2019. You want to sell 1 unit of European Call on AAPL, with strike of $180 and maturity of 1 year.



Suppose the annualized interest rate is 3%, the annualized drift for AAPL is 5%, no dividend, and the annualized volatility is 10%. In order to hedge the potential risk of your option, you are going to calculate several Greeks based on Black-Merton-Scholes model.



In [293]:
import numpy as np
from numpy import log, exp, sqrt
from scipy import stats
from typing import Tuple

class BSMOptionValuation:
    """
    Valuation of European call options in Black-Scholes-Merton Model (incl. dividend)
    Attributes
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time to maturity (in year fractions)
    r: float
        constant risk-free short rate
        assume flat term structure
    sigma: float
        volatility factor in diffusion term
    div_yield: float
        dividend_yield, in percentage %, default = 0.0%
    """

    def __init__(self, S0: float, K: float, T: float, r: float, sigma: float, div_yield: float = 0.0):
        assert sigma >= 0, 'volatility cannot be less than zero'
        assert S0 >= 0, 'initial stock price cannot be less than zero'
        assert T >= 0, 'time to maturity cannot be less than zero'
        assert div_yield >= 0, 'dividend yield cannot be less than zero'

        self.S0 = float(S0)
        self.K = float(K)
        self.T = float(T)
        self.r = float(r)
        self.sigma = float(sigma)
        self.div_yield = float(div_yield)

        self._d1, self._d2 = self._calculate_d1_d2()
        self._d3 = None
        self._d4 = None
        self._d5 = None
        self._d6 = None
        self._d7 = None
        self._d8 = None

    def _calculate_d1_d2(self):
        d1 = ((log(self.S0 / self.K) + (self.r - self.div_yield + 0.5 * self.sigma ** 2) * self.T) / (
                self.sigma * sqrt(self.T)))
        d2 = d1 - self.sigma * sqrt(self.T)

        return d1, d2

    def call_value(self, observed_put_price: float = None) -> float:
        """
        :return: call option value
        """
        if observed_put_price is None:
            call_value = (self.S0 * exp(-self.div_yield * self.T) * stats.norm.cdf(self._d1, 0.0, 1.0) - self.K * exp(
                -self.r * self.T) * stats.norm.cdf(self._d2, 0.0, 1.0))
        else:
            call_value = observed_put_price + exp(-self.div_yield * self.T) * self.S0 - exp(-self.r * self.T) * self.K

        return call_value

    def delta(self) -> Tuple[float, float]:
        """
        Delta measures the change in the option price for a $1 change in the stock price
        :return: delta of the option
        """
        delta_call = exp(- self.div_yield * self.T) * stats.norm.cdf(self._d1, 0.0, 1.0)
        delta_put = -exp(- self.div_yield * self.T) * stats.norm.cdf(-self._d1, 0.0, 1.0)

        return delta_call, delta_put

    def gamma(self) -> float:
        """
        Gamma measures the change in delta when the stock price changes
        :return: gamma of the option
        """
        gamma = exp(-self.div_yield * self.T) * stats.norm.pdf(self._d1) / (self.S0 * self.sigma * sqrt(self.T))

        return gamma

    def theta(self) -> Tuple[float, float]:
        """
        Theta measures the change in the option price with respect to calendar time (t ),
        holding fixed time to expiration (T).

        If time to expiration is measured in years, theta will be the annualized change in the option value.
        To obtain a per-day theta, divide by 252.
        :return: theta of the option
        """
        part1 = self.div_yield * self.S0 * exp(-self.div_yield * self.T) * stats.norm.cdf(self._d1)
        part2 = self.r * self.K * exp(-self.r * self.T) * stats.norm.cdf(self._d2)
        part3 = (self.S0 * exp(-self.div_yield * self.T) * stats.norm.pdf(self._d1) * self.sigma) / (2 * sqrt(self.T))
        # print(part3, exp(-self.div_yield * self.T), part1)
        theta_call = part1 - part2 - part3
        theta_put = theta_call + self.r * self.K * exp(-self.r * self.T) - self.div_yield * self.S0 * exp(
            -self.div_yield * self.T)

        return theta_call, theta_put

    def vega(self) -> float:
        """
        Vega measures the change in the option price when volatility changes. Some writers also
        use the terms lambda or kappa to refer to this measure:
        It is common to report vega as the change in the option price per percentage point change
        in the volatility. This requires dividing the vega formula above by 100.
        :return: vega of option
        """
        vega = self.S0 * exp(-self.div_yield * self.T) * stats.norm.pdf(self._d1, 0.0, 1.0) * sqrt(self.T)

        return vega
    def put_value(self, observed_call_price: float = None) -> float:
        """
        Use put call parity (incl. continuous dividend) to calculate the put option value

        :return: put option value
        """
        if observed_call_price is None:
            put_value = self.call_value() + exp(-self.r * self.T) * self.K - exp(-self.div_yield * self.T) * self.S0
        else:
            put_value = observed_call_price + exp(-self.r * self.T) * self.K - exp(-self.div_yield * self.T) * self.S0

        return put_value

    def implied_vol(self, observed_call_price: float, num_iterations: int = 1000, tolerance: float = 1e-4) -> float:
        """
        Newton-Raphson iterative approach, assuming black_scholes_merton model
        :param observed_call_price: call price from the market
        :param num_iterations: no. of iteration
        :param tolerance: allows to specify the tolerance level
        :return: implied volatility given the observed option price
        """
        sigma_old = self.sigma

        for _ in range(num_iterations):
            self._d1, self._d2 = self._calculate_d1_d2()
            _cal_val = self.call_value()
            option_price_diff = _cal_val - observed_call_price

            _vega = self.vega()
            sigma_new = self.sigma - option_price_diff / (_vega + 1e-10)

            if abs(sigma_new - self.sigma) <= tolerance:
                break

            self.sigma = sigma_new

        implied_vol = self.sigma

        # restore back the status
        self.sigma = sigma_old
        self._d1, self._d2 = self._calculate_d1_d2()
        return implied_vol

In [294]:
# initialize parameters
S0 = 171.01  # e.g. spot price = 35
K = 180  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.03  # e.g. risk free rate = 1%
sigma = 0.1  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%

# initialize instance
bsm = BSMOptionValuation(S0, K, T, r, sigma, div_yield)
np.round(bsm.call_value(), 5)


5.21223

In [295]:
Long = True
if Long:
   delta = bsm.delta()[0]
else:
    delta = -1* bsm.delta()[0]

np.round(delta, 5)

0.43552

In [296]:
if Long:
   gamma = bsm.gamma()
else:
    gamma = -1* bsm.gamma()

np.round(gamma, 5)

0.02302

In [297]:
if Long:
   vega = bsm.vega()
else:
    vega = -1* bsm.vega()

np.round(vega, 5)

67.32994

In [298]:
if Long:
   theta = bsm.theta()[0]
else:
    theta = -1* bsm.theta()[0]

np.round(theta, 5)
# bsm.theta()

-5.44446

In [299]:
print(f"The AAPL Call option was short so the hedging will to buy {np.round(delta, 5)} AAPL shares")
print(f"The net cash flow is {bsm.call_value() - bsm.delta()[0]*S0}")

The AAPL Call option was short so the hedging will to buy 0.43552 AAPL shares
The net cash flow is -69.2653294783826


In [300]:
S1 = 180.2
T1 = 11/12
bsm1 = BSMOptionValuation(S1, K, T1, r, sigma, div_yield)
np.round(bsm.call_value(), 5)

5.21223

In [301]:
print(f"The AAPL stock has gone up and Sold Call option is bleeding so more {np.round(bsm1.delta()[0] - bsm.delta()[0], 4)} shares to be bought")

The AAPL stock has gone up and Sold Call option is bleeding so more 0.2001 shares to be bought


In [302]:
print(bsm1.call_value(), bsm.call_value(), bsm.delta()[0], (S1-S0))
print(bsm1.call_value() - bsm.call_value())
print(bsm.delta()[0] * (S1-S0))
old_val = bsm.call_value() - bsm.delta()[0]*S0
print(exp(r*1/12) - 1)
print((bsm.call_value() - bsm.delta()[0]*S0)*(exp(r*1/12) - 1))
new_val = bsm1.call_value() - bsm1.delta()[0] *S1
print(new_val - old_val)

9.62488111192711 5.212234921590209 0.43551584351776396 9.189999999999998
4.4126461903369005
4.00239060192825
0.0025031276057951857
-0.17337995834183853
-35.6432834708359


In [303]:
S0 = 171.01  # e.g. spot price = 35
K_180 = 180  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.03  # e.g. risk free rate = 1%
sigma = 0.1  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%
K_185 = 185
bsm_180 = BSMOptionValuation(S0, K_180, T, r, sigma, div_yield)
bsm_185 = BSMOptionValuation(S0, K_185, T, r, sigma, div_yield)

port_val = bsm_180.call_value() - bsm_185.call_value()
port_val



1.6742566533948562

In [304]:
sigma_180 = np.minimum(1, 18/180)
sigma_185 = np.minimum(1, 18/185)
print(sigma_180, sigma_185)

0.1 0.0972972972972973


In [305]:
bsm_180_sig_fun = BSMOptionValuation(S0, K_180, T, r, sigma_180, div_yield)
bsm_185_sig_fun = BSMOptionValuation(S0, K_185, T, r, sigma_185, div_yield)

port_val_sig_fun = bsm_180_sig_fun.call_value() - bsm_185_sig_fun.call_value()
port_val_sig_fun

1.8413563495907752

In [306]:
S0 = 171.01  # e.g. spot price = 35
K_180 = 180  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.03  # e.g. risk free rate = 1%
sigma = 0.1  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%
K_180_1 = 180.1
bsm_180 = BSMOptionValuation(S0, K_180, T, r, sigma, div_yield)
bsm_180_1 = BSMOptionValuation(S0, K_180_1, T, r, sigma, div_yield)
d_180 = (bsm_180.call_value() - bsm_180_1.call_value())/(K_180_1-K_180)
d_180

0.38376904311365273

In [307]:
S0 = 171.01  # e.g. spot price = 35
K_185 = 185  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.03  # e.g. risk free rate = 1%
sigma = 0.1  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%
K_185_1 = 185.1
bsm_185 = BSMOptionValuation(S0, K_185, T, r, sigma, div_yield)
bsm_185_1 = BSMOptionValuation(S0, K_185_1, T, r, sigma, div_yield)
d_185 = (bsm_185.call_value() - bsm_185_1.call_value())/(K_185_1-K_185)
d_185

0.2862133177683453

In [308]:
d_180 - d_185

0.09755572534530743

In [309]:
S0 = 171.01  # e.g. spot price = 35
K_180 = 180  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.03  # e.g. risk free rate = 1%
sigma = sigma_180  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%
K_180_1 = 180.1
bsm_180 = BSMOptionValuation(S0, K_180, T, r, sigma, div_yield)
bsm_180_1 = BSMOptionValuation(S0, K_180_1, T, r, sigma, div_yield)
d_180 = (bsm_180.call_value() - bsm_180_1.call_value())/(K_180_1-K_180)
d_180

0.38376904311365273

In [310]:
S0 = 171.01  # e.g. spot price = 35
K_185 = 185  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.03  # e.g. risk free rate = 1%
sigma = sigma_185  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%
K_185_1 = 185.1
bsm_185 = BSMOptionValuation(S0, K_185, T, r, sigma, div_yield)
bsm_185_1 = BSMOptionValuation(S0, K_185_1, T, r, sigma, div_yield)
d_185 = (bsm_185.call_value() - bsm_185_1.call_value())/(K_185_1-K_185)
d_185

0.2821312634905287

In [311]:
d_180 - d_185

0.10163777962312404

In [312]:
S0 = 100  # e.g. spot price = 35
K = 90  # e.g. exercise price = 40
T = 1.0  # e.g. six months = 0.5
r = 0.05  # e.g. risk free rate = 1%
sigma = 0.1
div_yield = 0.0  # e.g. dividend yield = 1%
observed_call_price = 16.6994

bsm_new = BSMOptionValuation(S0, K, T, r, sigma, div_yield)
bsm_new.implied_vol(observed_call_price=observed_call_price) * 100

20.003507456799635

In [313]:
sigma = 0.3 * exp(-2*((K/100)-1))
bsm_new_2 = BSMOptionValuation(S0, K, T, r, sigma, div_yield)
print(f"The put value is {bsm_new_2.put_value()}")
# Long 1 Put and short shares so portfolio value is
print(f"The portfolio value is {-1* bsm_new_2.put_value() + bsm_new_2.delta()[1] *S0}")

The put value is 7.474685924232887
The portfolio value is -34.66042344298449


In [314]:
bsm_new_2.delta()[1]

-0.271857375187516

In [315]:
sigma = sigma * 1.1
S0 = 95
bsm_new_3 = BSMOptionValuation(S0, K, T, r, sigma, div_yield)
print(bsm_new_3.put_value(), bsm_new_3.delta()[1])
print(f"The portfolio value is {-1* bsm_new_3.put_value() + bsm_new_2.delta()[1] * S0}")

10.199741636659198 -0.3228577199482915
The portfolio value is -36.02619227947322


In [316]:
-36.02619227947322 - (-34.66042344298449)

-1.3657688364887335

In [317]:
x = bsm_new_3.put_value()
sigma = 0.3 * exp(-2*((K/100)-1))
bsm_new_4 = BSMOptionValuation(S0, K, T, r, sigma, div_yield)
y = bsm_new_4.put_value()
print(y, x, y/x)

8.952211569908599 10.199741636659198 0.877690032631139


In [318]:
c_100 = 14.23125479
c_100_1 = 14.16035708
c_99_1 = 14.30227668
dif = 100.1-100
butterfly = (c_100_1 -2 *c_100 + c_99_1)/(dif**2)
butterfly

0.012418000000204177